# 🖥️ MAI-UI on T4: Optimized GUI Agent with vLLM

This notebook runs MAI-UI on Google Colab's free T4 GPU with optimized vLLM settings.

**Requirements:** Google Colab with T4 GPU (free tier works!)


In [ ]:
# Cell 1: Check GPU and Install Dependencies
import subprocess
import sys

print("=" * 60)
print("🔍 GPU DETECTION")
print("=" * 60)
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"\n✅ GPU: {gpu_name}")
    print(f"✅ Memory: {gpu_memory:.1f} GB")
else:
    print("❌ No GPU! Enable in Runtime -> Change runtime type")
    sys.exit(1)

# Install dependencies
print("\n📦 Installing dependencies...")
%pip install -q vllm>=0.6.0 pillow requests jinja2
print("✅ Dependencies installed!")


In [ ]:
# Cell 2: Initialize vLLM with T4-Optimized Config
from vllm import LLM, SamplingParams
import time

T4_CONFIG = {
    "model": "Tongyi-MAI/MAI-UI-2B",
    "trust_remote_code": True,
    "dtype": "half",
    "max_model_len": 2048,
    "gpu_memory_utilization": 0.90,
    "enforce_eager": True,
    "max_num_seqs": 4,
    "limit_mm_per_prompt": {"image": 1, "video": 0},
    "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 512000},
}

print("🚀 Initializing vLLM (downloading ~4GB model)...")
init_start = time.time()
llm = LLM(**T4_CONFIG)
print(f"✅ Engine initialized in {time.time() - init_start:.1f}s")

# Memory check
allocated = torch.cuda.memory_allocated() / (1024**3)
print(f"📊 GPU Memory: {allocated:.2f} GB used")


In [ ]:
# Cell 3: Define MAI-UI Prompt Format
import re
import json

MAI_GROUNDING_PROMPT = """You are a GUI grounding agent. Given a screenshot and instruction, locate the UI element.
Output: <grounding_think>[reasoning]</grounding_think><answer>{"coordinate": [x, y]}</answer>
Coordinates: [0, 999] range where (0,0)=top-left."""

def build_prompt(instruction):
    return (f"<|im_start|>system\n{MAI_GROUNDING_PROMPT}<|im_end|>\n"
            f"<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>{instruction}<|im_end|>\n"
            "<|im_start|>assistant\n")

def parse_response(text):
    result = {"thinking": None, "coordinate": None, "raw": text}
    think = re.search(r"<grounding_think>(.*?)</grounding_think>", text, re.DOTALL)
    if think: result["thinking"] = think.group(1).strip()
    answer = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL)
    if answer:
        try:
            data = json.loads(answer.group(1).strip())
            if "coordinate" in data:
                result["coordinate"] = [data["coordinate"][0]/999, data["coordinate"][1]/999]
        except: pass
    return result

print("✅ Prompt functions defined")


In [ ]:
# Cell 4: Create Test Screenshot
from PIL import Image, ImageDraw

def create_test_image():
    img = Image.new('RGB', (1080, 1920), '#f5f5f5')
    draw = ImageDraw.Draw(img)
    draw.rectangle([0, 0, 1080, 80], fill='#1976D2')
    draw.text((40, 30), "9:41", fill='white')
    draw.rectangle([0, 80, 1080, 200], fill='#2196F3')
    draw.text((40, 120), "Settings", fill='white')
    for label, y in [("Wi-Fi", 280), ("Bluetooth", 400), ("Cellular", 520)]:
        draw.rectangle([0, y, 1080, y+100], fill='white', outline='#e0e0e0')
        draw.text((40, y+35), label, fill='#333')
        draw.ellipse([960, y+30, 1020, y+70], fill='#4CAF50')
    draw.rectangle([0, 1800, 1080, 1920], fill='white')
    for i, label in enumerate(["Home", "Search", "Settings", "Profile"]):
        draw.text((60 + i*270, 1840), label, fill='#666')
    return img

test_image = create_test_image()
print(f"📸 Created test image: {test_image.size}")

# Display
thumb = test_image.copy()
thumb.thumbnail((250, 400))
display(thumb)


In [ ]:
# Cell 5: Run MAI-UI Grounding Inference
sampling_params = SamplingParams(temperature=0.0, max_tokens=512, stop=["<|im_end|>"])

instructions = ["Click on Wi-Fi", "Click on Bluetooth", "Click on Settings", "Click on Home"]
results = []

print("🤖 Running inference...")
for i, inst in enumerate(instructions, 1):
    prompt = build_prompt(inst)
    inputs = {"prompt": prompt, "multi_modal_data": {"image": test_image}}
    
    start = time.time()
    outputs = llm.generate([inputs], sampling_params=sampling_params)
    latency = (time.time() - start) * 1000
    
    parsed = parse_response(outputs[0].outputs[0].text)
    results.append({"instruction": inst, "latency_ms": latency, "parsed": parsed})
    
    if parsed["coordinate"]:
        x, y = parsed["coordinate"]
        print(f"[{i}] {inst}: ({x:.3f}, {y:.3f}) - {latency:.0f}ms")
    else:
        print(f"[{i}] {inst}: No coordinate - {latency:.0f}ms")

print("✅ Inference complete!")


In [ ]:
# Cell 6: Visualize Results
def visualize_clicks(image, results):
    img = image.copy()
    draw = ImageDraw.Draw(img)
    colors = ['#FF0000', '#00FF00', '#0000FF', '#FF00FF']
    for i, r in enumerate(results):
        coord = r["parsed"].get("coordinate")
        if coord:
            x, y = int(coord[0] * image.width), int(coord[1] * image.height)
            draw.ellipse([x-25, y-25, x+25, y+25], outline=colors[i], width=4)
            draw.line([x-35, y, x+35, y], fill=colors[i], width=3)
            draw.line([x, y-35, x, y+35], fill=colors[i], width=3)
    return img

vis = visualize_clicks(test_image, results)
vis_thumb = vis.copy()
vis_thumb.thumbnail((300, 500))
display(vis_thumb)

print("\n📊 Summary:")
for i, r in enumerate(results, 1):
    coord = r["parsed"].get("coordinate", "N/A")
    print(f"  [{i}] {r['instruction']}: {coord}")
